In [ ]:
import geopandas as gpd
import datetime

import sys
sys.path.append('..')

import rsutils.s2_grid_utils
import create_datacube

In [ ]:
rsutils.s2_grid_utils.grid_size_2_res_mapping(grid_size_km = 5)

In [ ]:
s2_grid_id = rsutils.s2_grid_utils.get_id_from_latlon(
    lat = 15.941989,
    lon = 77.157222,
    grid_size_km = 5,
    # res = 11,
)
s2_grid_id

In [ ]:
# override
s2_grid_id = '164c59c' # '164c59c' 

In [ ]:
s2grid_geom = rsutils.s2_grid_utils.get_grid_geometry_from_id(grid_id = s2_grid_id)
s2grid_geom

In [ ]:
s2_grid_gdf = gpd.GeoDataFrame(
    data = {
        'geometry': [s2grid_geom],
    },
    crs = 'epsg:4326',
)

# s2_grid_gdf.to_file(f'../data/{s2_grid_id}.geojson')

In [ ]:
catalog_filepath = '../data/cluster_files/satellite/Sentinel-2/catalog.geojson'
shapes_gdf = s2_grid_gdf
startdate = datetime.datetime(2019,3,1)
enddate = datetime.datetime(2022,1,31)

In [ ]:
stats = create_datacube.query_catalog_stats(
    catalog_filepath = catalog_filepath,
    shapes_gdf = shapes_gdf,
    startdate = startdate,
    enddate = enddate,
)

stats

In [ ]:
_, _, msg = create_datacube.check_if_there_are_files_missing(
    catalog_filepath = catalog_filepath,
    shapes_gdf = shapes_gdf,
    startdate = startdate,
    enddate = enddate,
    files = ['B01.jp2', 'VV.tif']
)
print(msg)

In [ ]:
import numpy as np

In [ ]:
stats.keys()

In [ ]:
min_timestamp = stats['timestamp_range'][0]

In [ ]:
fmt = '%Y%m%dT%H%M%S'
'_'.join([ts.strftime(fmt) for ts in stats['timestamp_range']])

In [ ]:
import shapely

A = 38.09192035575927
B = 8.141009568663828
C = 39.08858871535991
D = 8.142015435187918
E = 39.088384163693355
F = 7.148816474425373
G = 38.09401676842014
H = 7.147934625992314

diff = 1e-16

shapely.MultiPolygon([
    shapely.Polygon([
        (A, B + diff), 
        (C, D), 
        (E, F), 
        (G, H), 
        (A, B + diff),
    ])
]).buffer(0) == shapely.Polygon([
    (A, B), 
    (C, D), 
    (E, F), 
    (G, H), 
    (A, B),
]).buffer(0)

In [ ]:
diff = 1e-10

shapely.equals(
    shapely.MultiPolygon([
        shapely.Polygon([
            (A, B + diff), 
            (C, D), 
            (E, F), 
            (G, H), 
            (A, B + diff),
        ])
    ]),
    shapely.Polygon([
        (A, B), 
        (C, D), 
        (E, F), 
        (G, H), 
        (A, B),
    ])
)

In [ ]:
east_bb_eth_gdf = gpd.read_file('../../ethiopia/data/Ethiopia_Bounding_Boxs/Ethiopia_2021_East_Bounding_Box.shp')

In [ ]:
east_bb_s2grids_gdf = rsutils.s2_grid_utils.get_s2_grids_gdf(
    geojson_epsg_4326 = east_bb_eth_gdf.to_crs('epsg:4326')['geometry'][0],
)

In [ ]:
east_bb_s2grids_gdf.to_file('../data/eth_east_bb_s2grids.geojson')